In [ ]:
#First Lets Create the entropy and apply one hot encoding to all categories


In [1]:
#import block

import pandas as pd


In [ ]:
tab_data_unfiltered = pd.read_csv("data\motionStrike_TVcodes_data.tsv", delimiter = '\t')
tab_data_unfiltered.head()

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\aarya\AppData\Local\Temp\ipykernel_3976\3367256887.py:1: SyntaxWarning: invalid escape sequence '\m'
  tab_data_unfiltered = pd.read_csv("data\motionStrike_TVcodes_data.tsv", delimiter = '\t')


,CaseReferenceNumber,MotionID,CaseLocation,CaseMajorCode,CaseMinorCode,CaseTrialListType,CaseMarkingCode,CaseDispositionJudgeJurisNo,CaseDispositionDocketLegendCode,MotionResultCode,...,MotionJurisNumber,MotionDocumentPriorityCode,CaseAttorneyJuris,CaseAttorneyType,MotionTimeDuration,MotionDocumentTypeName,SelfRepBeforeMotionFileCount,SelfRepBeforeMotionDecidedCount,SelfRepBeforeMotionFileBool,SelfRepBeforeMotionDecidedBool
0,207,1751,FBT,V,1,9,16.0,416586.0,JDGDACT,GR,...,418027.0,30,418027.0,J,78.0,MOTION TO STRIKE,1,1,True,True
1,1472,10915,FBT,V,1,9,16.0,NaN,WDACT,DN,...,416586.0,30,416586.0,R,22.0,MOTION TO STRIKE,0,0,False,False
2,1494,11050,NNI,V,1,9,16.0,NaN,WDACT,GR,...,422392.0,30,422392.0,J,285.0,MOTION TO STRIKE,0,0,False,False
3,371,2708,LLI,T,90,-1,NaN,418028.0,SJD,OR,...,418028.0,30,418028.0,R,120.0,MOTION TO STRIKE,0,0,False,False
4,1422,10373,HHD,T,2,-1,NaN,422394.0,SJ,GR,...,81181.0,30,81181.0,R,153.0,MOTION TO STRIKE,0,0,False,False


In [8]:
#important columns identifiers 
target= tab_data_unfiltered["MotionResultCode"]
#potential features
features = tab_data_unfiltered[["CaseMajorCode", "MotionTimeDuration", "CaseLocation"]]


In [ ]:
#for case location find each value like FBT NNI assign it to a key and then replace the key 
count_caseLoc = {}
for i in features["CaseLocation"]:
    count_caseLoc[i] = count_caseLoc.get(i, 0) + 1

Caselength = len(count_caseLoc)
print(f"The Length is {Caselength} and the frequencies are:\n {count_caseLoc}")
#assign values 1 to 15


#binary encoding to CaseMajorCode 

#for CaseLocation two methods
#label encoding or frequency encoding
#less variability with near by number allocation with frequency encoding try that




The Length is 15 and the frequencies are:
 {'FBT': 2003, 'NNI': 213, 'LLI': 287, 'HHD': 3094, 'FST': 1494, 'MMX': 519, 'NNH': 1528, 'KNO': 67, 'HHB': 999, 'KNL': 833, 'DBD': 489, 'UWY': 1369, 'AAN': 518, 'TTD': 183, 'WWM': 207}


In [9]:
#entropy filtering for each column
features.head()



,CaseMajorCode,MotionTimeDuration,CaseLocation
0,V,78.0,FBT
1,V,22.0,FBT
2,V,285.0,NNI
3,T,120.0,LLI
4,T,153.0,HHD
